In [1]:
import pandas  as pd
import numpy as np

In [2]:
writers=pd.read_csv('/Users/macbook/Desktop/BA6/Database/db2018imdb/writers.csv', sep=',')
people_old=pd.read_csv('/Users/macbook/Desktop/BA6/Database/newfiles/people.csv', sep=',')

In [3]:
cols_to_check = ['ClipIds','Roles','AddInfos', 'WorkTypes']
writers[cols_to_check] = writers[cols_to_check].replace({'\[':'','\]':''},regex=True)

writers = writers.replace('', np.nan, regex=True)

writers.drop_duplicates(keep='first', inplace=True)
writers.drop_duplicates(subset=['FullName'], keep='first', inplace=True)

In [4]:
result = pd.merge(writers, people_old, how='left', on='FullName')
result = result[np.isnan(result['pid'])]
result['pid']=(result.FullName).rank(method='dense')+people_old['pid'].max()
result[['pid']] = result[['pid']].astype(int)
result= result[['pid','FullName']]
people = pd.concat([people_old, result])

In [5]:
people.to_csv('people.csv', sep=',', encoding='utf-8', index=False)

In [6]:
writers = pd.merge(writers, people, how='left', on='FullName')

In [7]:
write = writers.drop(['FullName'],axis=1)

write[['ClipIds']] = write[['ClipIds']].astype(str)
write[['AddInfos']] = write[['AddInfos']].astype(str)
write[['Roles']] = write[['Roles']].astype(str)
write[['WorkTypes']] = write[['WorkTypes']].astype(str)

def splitDataFrameList(df,target_column,separator):
    ''' df = dataframe to split,
    target_column = the column containing the values to split
    separator = the symbol used to perform the split
    returns: a dataframe with each entry for the target column separated, with each element moved into a new row. 
    The values in the other columns are duplicated across the newly divided rows.
    '''
    def splitListToRows(row,row_accumulator,target_column,separator):
        split_row = []
        i=0
        for c in target_column:
            split_row.append(row[c].split(separator))
            i+=1
            
        for s in range (0,len(split_row[0])):
            j=0
            new_row = row.to_dict()
            for c in target_column:
                new_row[c] = split_row[j][s]
                j+=1
            row_accumulator.append(new_row)
    new_rows = []
    df.apply(splitListToRows,axis=1,args = (new_rows,target_column,separator))
    global new_df
    new_df = pd.DataFrame(new_rows)
    return new_df

write = splitDataFrameList( write, ['ClipIds','AddInfos','Roles','WorkTypes'] , '|')

In [8]:
write = write.replace('', np.nan, regex=True)
write = write.replace('nan', np.nan, regex=True)

write = write[['pid', 'ClipIds', 'Roles', 'AddInfos','WorkTypes']]

write.drop_duplicates(subset=['pid','ClipIds'], keep=False)

,pid,ClipIds,Roles,AddInfos,WorkTypes
0,2006887,51264,NaN,written for the screen by,NaN
1,2006887,742507,writer,NaN,NaN
2,2006887,960777,written by,NaN,NaN
3,54,4016759,NaN,2000,NaN
4,59,338267,NaN,"novel ""Die Sonnenuhr""",NaN
5,59,437829,NaN,as Maarten Hart,novel
6,59,623582,NaN,NaN,novel
7,59,781970,NaN,"novel ""De kroongetuige"" as Maarten Hart",NaN
8,59,3347664,writer,NaN,NaN
9,65,2118411,writer,NaN,NaN


In [9]:
write.to_csv('write.csv', sep=',', encoding='utf-8', index=False)